<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# xDeepFM : the eXtreme Deep Factorization Machine 
このノートブックでは、[xDeepFM モデル](https://arxiv.org/abs/1803.05170)をトレーニングする方法の簡単な例を示します。
xDeepFM [1] は、低次から高次のフィーチャ インタラクションの両方をキャプチャすることを目的とした、ディープラーニングベースのモデルを使用した精密なレコメンデーション システムです。したがって、フィーチャーの相互作用をより効果的に学習でき、手動でのフィーチャー エンジニアリングの労力を大幅に削減できます。要約すると、xDeepFM には次の主要な要素があります:
* CIN という名前のコンポーネントが含まれており、フィーチャの相互作用を明示的な方法でベクトル単位で学習します。
* 暗黙的な方法でビット単位でフィーチャーの相互作用を学習する、従来の DNN コンポーネントが含まれています。
* このモデルは非常に構成変更が可能な実装です。`use_Linear_part`、 `use_FM_part`、 `use_CIN_part` 及び `use_DNN_part` などのハイパーパラメーターを設定することで、コンポーネントの異なるサブセットを有効にできます。たとえば、`use_Linear_part` と `use_FM_part` のみを有効にするとで、古典的な FM モデルを取得できます。

このノートブックでは、1) 小さな合成データセットと 2) [Criteo データセット](http://labs.criteo.com/category/dataset)の 2 つのデータセットで xDeepFM をテストできます。

## 0. グローバル設定とインポート

In [1]:
import sys
sys.path.append("../../")
import os
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from reco_utils.common.constants import SEED
from reco_utils.recommender.deeprec.deeprec_utils import (
    download_deeprec_resources, prepare_hparams
)
from reco_utils.recommender.deeprec.models.xDeepFM import XDeepFMModel
from reco_utils.recommender.deeprec.io.iterator import FFMTextIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))


System version: 3.6.11 | packaged by conda-forge | (default, Aug  5 2020, 20:09:42) 
[GCC 7.5.0]
Tensorflow version: 1.15.2


#### パラメータ

In [1]:
EPOCHS_FOR_SYNTHETIC_RUN = 15
EPOCHS_FOR_CRITEO_RUN = 10
BATCH_SIZE_SYNTHETIC = 128
BATCH_SIZE_CRITEO = 4096
RANDOM_SEED = SEED  # 非確定的な結果にする際には None を設定

xDeepFM はデータ入力として 次のようなFFM 形式を使用します: `<label> <field_id>:<feature_id>:<feature_value>`
各行はインスタンスを表し、`<label>` では 1 は正のインスタンスを意味し、0 は負のインスタンスを意味するバイナリ値です。
フィーチャはフィールドに分割されます。例えば、ユーザの性別はフィールドであり、男性、女性および不明の3つの選択可能な値を含みます。職業は、性別フィールドよりも多くの選択可能な値を含む別のフィールドにすることができます。フィールド インデックスとフィーチャ インデックスの両方が 1 から始まります。<br>

## 1.合成データ
それでは、小さな合成データセットから始めましょう。このデータセットには、10 のフィールド、1000 のフィーチャーがあり、一連の事前設定されたペアごとのフィーチャの相互作用の結果に従ってラベルが生成されます。

In [3]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'xDeepFM.yaml')
train_file = os.path.join(data_path, r'synthetic_part_0')
valid_file = os.path.join(data_path, r'synthetic_part_1')
test_file = os.path.join(data_path, r'synthetic_part_2')
output_file = os.path.join(data_path, r'output.txt')

if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/deeprec/', data_path, 'xdeepfmresources.zip')


100%|██████████| 10.3k/10.3k [00:01<00:00, 5.24kKB/s]


#### 1.1 ハイパーパラメータの準備
prepare_hparams() は、学習率、フィーチャ数、ドロップアウト率など、モデルトレーニング用のハイパーパラメータの完全なセットを作成します。これらのパラメータを yaml ファイルに入れたり、関数のパラメーターとしてパラメータを渡したりできます (yaml 設定を上書きします)。

In [4]:
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=1000, 
                          FIELD_COUNT=10, 
                          cross_l2=0.0001, 
                          embed_l2=0.0001, 
                          learning_rate=0.001, 
                          epochs=EPOCHS_FOR_SYNTHETIC_RUN,
                          batch_size=BATCH_SIZE_SYNTHETIC)
print(hparams)

kg_file=None,user_clicks=None,FEATURE_COUNT=1000,FIELD_COUNT=10,data_format=ffm,PAIR_NUM=None,DNN_FIELD_NUM=None,n_user=None,n_item=None,n_user_attr=None,n_item_attr=None,iterator_type=None,SUMMARIES_DIR=None,MODEL_DIR=None,wordEmb_file=None,entityEmb_file=None,contextEmb_file=None,news_feature_file=None,user_history_file=None,use_entity=True,use_context=True,doc_size=None,history_size=None,word_size=None,entity_size=None,entity_dim=None,entity_embedding_method=None,transform=None,train_ratio=None,dim=10,layer_sizes=[100, 100],cross_layer_sizes=[1],cross_layers=None,activation=['relu', 'relu'],cross_activation=identity,user_dropout=False,dropout=[0.0, 0.0],attention_layer_sizes=None,attention_activation=None,attention_dropout=0.0,model_type=xDeepFM,method=classification,load_saved_model=False,load_model_name=you model path,filter_sizes=None,num_filters=None,mu=None,fast_CIN_d=0,use_Linear_part=False,use_FM_part=False,use_CIN_part=True,use_DNN_part=False,init_method=tnormal,init_value=0

#### 1.2 データローダーの作成
モデルのデータ用イテレータを指定します。xDeepFM は FFMTextIterator を使用します。

In [5]:
input_creator = FFMTextIterator

#### 1.3 モデルの作成
ハイパーパラメータとデータ イテレータの両方の準備ができたら、モデルを作成できます。

In [6]:
model = XDeepFMModel(hparams, input_creator, seed=RANDOM_SEED)

## モデルを最初からトレーニングしたくない場合は、次のような
## 事前トレーニング済みのモデルを読み込むことができます:
#model.load_model(r'your_model_path')

Add CIN part.


次に、この時点でのモデルのパフォーマンスを見てみましょう (トレーニングを開始せずに):

In [7]:
print(model.run_eval(test_file))

{'auc': 0.5043, 'logloss': 0.7515}


AUC=0.5 はランダム推論の状態です。トレーニングの前に、モデルがランダムな推測のように振る舞うことが確認できます。

#### 1.4 モデルのトレーニング
次に、トレーニング セットでモデルをトレーニングし、検証データセットのパフォーマンスを確認します。モデルのトレーニングは、関数呼び出しと同じくらい簡単です:

In [8]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:0.7556826068773302
eval info: auc:0.504, logloss:0.7042
at epoch 1 , train time: 4.3 eval time: 0.6
at epoch 2
train info: logloss loss:0.7263523231666932
eval info: auc:0.5066, logloss:0.6973
at epoch 2 , train time: 4.3 eval time: 0.8
at epoch 3
train info: logloss loss:0.7177084291104189
eval info: auc:0.5099, logloss:0.6953
at epoch 3 , train time: 3.8 eval time: 0.7
at epoch 4
train info: logloss loss:0.7118660186983875
eval info: auc:0.5147, logloss:0.6946
at epoch 4 , train time: 3.7 eval time: 0.8
at epoch 5
train info: logloss loss:0.7055103289302682
eval info: auc:0.523, logloss:0.6941
at epoch 5 , train time: 3.8 eval time: 0.7
at epoch 6
train info: logloss loss:0.6954095556154284
eval info: auc:0.5416, logloss:0.6929
at epoch 6 , train time: 3.6 eval time: 0.8
at epoch 7
train info: logloss loss:0.6723950118133702
eval info: auc:0.5916, logloss:0.6831
at epoch 7 , train time: 4.0 eval time: 0.7
at epoch 8
train info: logloss loss:0.61198

#### 1.5 モデルの評価

ここでも、モデルのパフォーマンスを見てみましょう (トレーニング後):

In [9]:
res_syn = model.run_eval(test_file)
print(res_syn)


{'auc': 0.9716, 'logloss': 0.2278}


In [10]:
sb.glue("res_syn", res_syn)

評価メトリックではなく完全な予測スコアを取得する場合は、次の操作で行うことができます:

In [11]:
model.predict(test_file, output_file)

## 2. Criteo データ 

これで、合成データセットの実験に成功しました。次に、実世界のデータセットとして [Criteo データセット](http://labs.criteo.com/category/dataset)の小さなサンプルで何かを試してみましょう。Criteo データセットは、CTR 予測モデルを開発するための業界のベンチマーク データセットとしてよく知られており、研究論文による評価データセットとして頻繁に採用されています。元のデータセットは軽量のデモでは大きすぎるため、デモ データセットとしてそのデータセットの一部をサンプリングします。

In [12]:
print('Criteo データセットによるデモ')
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=2300000, 
                          FIELD_COUNT=39, 
                          cross_l2=0.01, 
                          embed_l2=0.01, 
                          layer_l2=0.01,
                          learning_rate=0.002, 
                          batch_size=BATCH_SIZE_CRITEO, 
                          epochs=EPOCHS_FOR_CRITEO_RUN, 
                          cross_layer_sizes=[20, 10], 
                          init_value=0.1, 
                          layer_sizes=[20,20],
                          use_Linear_part=True, 
                          use_CIN_part=True, 
                          use_DNN_part=True)



demo with Criteo dataset


In [13]:
train_file = os.path.join(data_path, r'cretio_tiny_train')
valid_file = os.path.join(data_path, r'cretio_tiny_valid')
test_file = os.path.join(data_path, r'cretio_tiny_test')

In [14]:
model = XDeepFMModel(hparams, FFMTextIterator, seed=RANDOM_SEED)

Add linear part.
Add CIN part.
Add DNN part.


In [15]:
# モデルをトレーニングする前に予測パフォーマンスを確認する
print(model.run_eval(test_file)) 


{'auc': 0.4728, 'logloss': 0.7113}


In [16]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:744.3602027893066
eval info: auc:0.6637, logloss:0.5342
at epoch 1 , train time: 21.7 eval time: 4.3
at epoch 2
train info: logloss loss:385.66927337646484
eval info: auc:0.7137, logloss:0.5109
at epoch 2 , train time: 21.4 eval time: 4.3
at epoch 3
train info: logloss loss:191.50830841064453
eval info: auc:0.7283, logloss:0.5037
at epoch 3 , train time: 21.4 eval time: 4.2
at epoch 4
train info: logloss loss:92.20774269104004
eval info: auc:0.7359, logloss:0.4991
at epoch 4 , train time: 21.6 eval time: 4.4
at epoch 5
train info: logloss loss:43.159456968307495
eval info: auc:0.74, logloss:0.4963
at epoch 5 , train time: 21.6 eval time: 4.3
at epoch 6
train info: logloss loss:19.656921446323395
eval info: auc:0.7426, logloss:0.4946
at epoch 6 , train time: 21.3 eval time: 4.2
at epoch 7
train info: logloss loss:8.77035716176033
eval info: auc:0.7441, logloss:0.4934
at epoch 7 , train time: 21.5 eval time: 4.3
at epoch 8
train info: logloss loss:3.92

In [17]:
# モデルをトレーニングした後の予測パフォーマンスを確認する
res_real = model.run_eval(test_file)
print(res_real)

{'auc': 0.7356, 'logloss': 0.5017}


In [18]:
sb.glue("res_real", res_real)

In [19]:
# クリーンアップ
tmpdir.cleanup()

## 参考文献
\[1\] Lian, J., Zhou, X., Zhang, F., Chen, Z., Xie, X., & Sun, G. (2018). xDeepFM: Combining Explicit and Implicit Feature Interactions for Recommender Systems. Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery \& Data Mining, KDD 2018, London, UK, August 19-23, 2018.<br>